In [2]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

diretorio_imagens = './data/patches/'
diretorio_caracteristicas = './data/caracteristicas/'
arquivo_csv = './data/caracteristicas/LBP.csv'

# Se o diretório não existir, cria
if not os.path.exists(diretorio_caracteristicas):
    os.makedirs(diretorio_caracteristicas)

def get_pixel(img, center, x, y):
    new_value = 0
    try:
        if img[x][y] >= center:
            new_value = 1
    except:
        pass
    return new_value

def lbp_calculated_pixel(img, x, y):
    '''
    64 | 128 |   1
    ----------------
    32 |   0 |   2
    ----------------
    16 |   8 |   4    
    '''    
    center = img[x][y]
    val_ar = []
    val_ar.append(get_pixel(img, center, x-1, y+1))     # top_right
    val_ar.append(get_pixel(img, center, x, y+1))       # right
    val_ar.append(get_pixel(img, center, x+1, y+1))     # bottom_right
    val_ar.append(get_pixel(img, center, x+1, y))       # bottom
    val_ar.append(get_pixel(img, center, x+1, y-1))     # bottom_left
    val_ar.append(get_pixel(img, center, x, y-1))       # left
    val_ar.append(get_pixel(img, center, x-1, y-1))     # top_left
    val_ar.append(get_pixel(img, center, x-1, y))       # top
    
    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
    val = 0
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
    return val

# Inicialize um DataFrame vazio para armazenar as características LBP
df = pd.DataFrame()

# Percorra todos os diretórios [key]
for key in os.listdir(diretorio_imagens):
    key_path = os.path.join(diretorio_imagens, key)
    
    # Percorra todas as imagens no diretório [key]
    for image_name in os.listdir(key_path):
        image_path = os.path.join(key_path, image_name)
        
        # Carregue a imagem em tons de cinza
        imagem = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        # Calcule as características LBP
        height, width = imagem.shape
        img_lbp = np.zeros((height, width), np.uint8)
        for i in range(height):
            for j in range(width):
                img_lbp[i, j] = lbp_calculated_pixel(imagem, i, j)

        # Obtenha o nome do arquivo original
        nome_arquivo_original = image_name.split('_')[0]
        
        # Adicione as características e o nome do arquivo ao DataFrame
        lbp_features = img_lbp.ravel()
        df = df.append(pd.Series([key, nome_arquivo_original] + list(lbp_features)), ignore_index=True)

        # Exiba a imagem com as texturas
        # plt.imshow(img_lbp, cmap='gray')
        # plt.show()

        print(f'Processando {image_path}')

# Defina os nomes das colunas no DataFrame
colunas = ['key', 'nome_arquivo'] + [f'LBP_{i}' for i in range(len(lbp_features))]
df.columns = colunas

# Salve o DataFrame em um arquivo CSV
df.to_csv(arquivo_csv, index=False)


C:\Users\Usuario\AppData\Local\Temp\ipykernel_4972\86738341.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series([key, nome_arquivo_original] + list(lbp_features)), ignore_index=True)


Processando ./data/patches/abethr1\XC128013_patch_0.png
Processando ./data/patches/abethr1\XC128013_patch_1.png
Processando ./data/patches/abethr1\XC128013_patch_2.png
Processando ./data/patches/abethr1\XC128013_patch_3.png
Processando ./data/patches/abethr1\XC128013_patch_4.png
Processando ./data/patches/abethr1\XC128013_patch_5.png


KeyboardInterrupt: 